In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
from scipy import stats
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6.5.2/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri, Dataset):
    Data_DF = Open_File(Diri+Model+'_Data_'+Dataset+'_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_'+Dataset+'_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
# Edit Time Format
    Time, Begin_Time, Compl_Time, Trop_Peak_Time, Peak_Time, Birth_Time, Death_Time, Year = [], [], [], [], [], [], [], []
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Trop_Peak_Time.append(Datetime(ET_DF["Trop Peak Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
        Birth_Time.append(Datetime(ET_DF["Genesis Time"][j]))
        Death_Time.append(Datetime(ET_DF["Dissipation Time"][j]))
        Year.append(Find_Year(ET_DF["ET Begin Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Trop Peak Time"] = Trop_Peak_Time
    ET_DF["Peak Time"] = Peak_Time
    ET_DF["Genesis Time"] = Birth_Time
    ET_DF["Dissipation Time"] = Death_Time
    ET_DF["Year"] = Year
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    try:
        New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    except:
        New_Time = numpy.nan
    return (New_Time)

In [6]:
def Find_Year(Time):
    try:
        New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
        Year = New_Time.year
    except:
        Year = numpy.nan
    return (Year)

In [7]:
# Create Phase Distribution Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)
Lat_Bins = Create_Bins(10,60,5)
Lon_Bins = Create_Bins(-100,20,5)

In [8]:
Num_Years = numpy.array([90,93,93])

In [9]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri, "SubsetB")
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri, "SubsetB")
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri, "SubsetB")

In [10]:
Control_Data_A, Control_ET_A, Control_Codes = Files_Open("Control", Diri, "SubsetA")
RCP45_Data_A, RCP45_ET_A, RCP45_Codes = Files_Open("RCP45", Diri, "SubsetA")
RCP85_Data_A, RCP85_ET_A, RCP85_Codes = Files_Open("RCP85", Diri, "SubsetA")

In [11]:
Control_ET

,Code,Name,ABC,Trans Type,ET Begin Time,ET Complete Time,Trop Peak Time,Peak Time,Genesis Time,Dissipation Time,...,Trop Peak SLP,Trop Peak Lon,Trop Peak Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat,Year
0,TC190001,Alex,A,3,1900-07-03 18:00:00,1900-07-03 18:00:00,1900-06-25 06:00:00,1900-06-25 06:00:00,1900-06-19 12:00:00,1900-07-04 00:00:00,...,987.41,-84.91,29.95,1015.57,-90.75,36.25,1015.57,-90.75,36.25,1900
1,TC190003,Colin,A,1,1900-09-18 00:00:00,1900-09-20 12:00:00,1900-09-15 18:00:00,1900-09-19 12:00:00,1900-09-09 18:00:00,1900-09-20 18:00:00,...,943.73,-77.15,24.55,952.06,-77.17,31.14,979.68,-62.68,47.61,1900
2,TC190004,Dawn,A,1,1900-11-05 12:00:00,1900-11-06 12:00:00,1900-11-04 00:00:00,1900-11-04 00:00:00,1900-10-29 00:00:00,1900-11-09 06:00:00,...,948.88,-91.58,20.70,954.74,-87.08,27.00,992.74,-74.44,30.88,1900
3,TC190005,Ernest,A,1,1900-11-06 00:00:00,1900-11-08 00:00:00,1900-11-05 00:00:00,1900-11-06 18:00:00,1900-10-31 18:00:00,1900-11-08 06:00:00,...,994.44,-32.46,35.66,996.02,-27.73,41.42,998.22,-9.00,45.00,1900
4,TC190102,Harrison,A,1,1901-05-28 12:00:00,1901-05-30 18:00:00,1901-05-27 06:00:00,1901-05-27 06:00:00,1901-05-23 00:00:00,1901-06-01 06:00:00,...,980.89,-51.83,25.92,991.56,-43.71,30.04,1008.78,-34.75,24.00,1901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,TC198802,Micah,C,1,1988-02-06 00:00:00,1988-02-06 06:00:00,1988-02-05 06:00:00,1988-02-05 06:00:00,1988-02-02 00:00:00,1988-02-07 12:00:00,...,998.09,-35.19,36.46,999.63,-31.01,38.29,999.02,-28.74,38.72,1988
278,TC198805,Seldoen,C,1,1988-09-07 06:00:00,1988-09-08 12:00:00,1988-09-07 00:00:00,1988-09-07 00:00:00,1988-08-29 06:00:00,1988-09-09 12:00:00,...,980.43,-49.24,32.21,984.68,-48.37,33.35,987.91,-35.17,42.45,1988
279,TC198808,Alice,C,2,1988-09-12 06:00:00,1988-09-12 12:00:00,1988-09-09 18:00:00,1988-09-13 06:00:00,1988-09-06 12:00:00,1988-09-13 06:00:00,...,998.54,-45.95,35.61,1009.08,-39.50,44.00,1008.35,-37.50,45.75,1988
280,TC198809,Bruce,C,1,1988-09-21 06:00:00,1988-09-27 00:00:00,1988-09-20 12:00:00,1988-09-22 00:00:00,1988-09-15 06:00:00,1988-09-27 18:00:00,...,961.85,-68.34,28.60,961.07,-66.39,30.40,1001.36,-41.00,47.00,1988


In [12]:
Control_ET_A

,Code,Name,ABC,Trans Type,ET Begin Time,ET Complete Time,Trop Peak Time,Peak Time,Genesis Time,Dissipation Time,...,Trop Peak SLP,Trop Peak Lon,Trop Peak Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat,Year
0,TC190001,Alex,A,3,1900-07-03 18:00:00,1900-07-03 18:00:00,1900-06-25 06:00:00,1900-06-25 06:00:00,1900-06-19 12:00:00,1900-07-04 00:00:00,...,987.41,-84.91,29.95,1015.57,-90.75,36.25,1015.57,-90.75,36.25,1900.0
1,TC190002,Beatrice,A,0,1900-08-17 06:00:00,NaT,1900-08-14 06:00:00,1900-08-14 06:00:00,1900-08-09 00:00:00,1900-08-18 12:00:00,...,887.54,-70.56,27.22,956.40,-62.06,44.15,NaN,NaN,NaN,1900.0
2,TC190003,Colin,A,1,1900-09-18 00:00:00,1900-09-20 12:00:00,1900-09-15 18:00:00,1900-09-19 12:00:00,1900-09-09 18:00:00,1900-09-20 18:00:00,...,943.73,-77.15,24.55,952.06,-77.17,31.14,979.68,-62.68,47.61,1900.0
3,TC190004,Dawn,A,1,1900-11-05 12:00:00,1900-11-06 12:00:00,1900-11-04 00:00:00,1900-11-04 00:00:00,1900-10-29 00:00:00,1900-11-09 06:00:00,...,948.88,-91.58,20.70,954.74,-87.08,27.00,992.74,-74.44,30.88,1900.0
4,TC190005,Ernest,A,1,1900-11-06 00:00:00,1900-11-08 00:00:00,1900-11-05 00:00:00,1900-11-06 18:00:00,1900-10-31 18:00:00,1900-11-08 06:00:00,...,994.44,-32.46,35.66,996.02,-27.73,41.42,998.22,-9.00,45.00,1900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,TC198810,Candy,C,0,1988-10-02 00:00:00,NaT,1988-09-21 18:00:00,1988-10-04 12:00:00,1988-09-18 06:00:00,1988-10-05 12:00:00,...,944.88,-37.45,17.34,957.08,-49.33,34.98,NaN,NaN,NaN,1988.0
645,TC198811,Dylan,C,0,1988-10-23 12:00:00,NaT,1988-10-12 00:00:00,1988-10-12 00:00:00,1988-09-22 12:00:00,1988-10-26 06:00:00,...,926.83,-52.84,30.49,993.78,-40.18,42.43,NaN,NaN,NaN,1988.0
646,TC198902,Ivy,C,1,1989-09-27 00:00:00,1989-09-27 18:00:00,1989-09-19 18:00:00,1989-09-19 18:00:00,1989-09-12 12:00:00,1989-09-29 18:00:00,...,953.02,-38.61,31.20,1000.11,-21.27,47.20,1007.69,-15.50,47.25,1989.0
647,TC198903,Jack,C,0,1989-09-21 12:00:00,NaT,1989-09-20 12:00:00,1989-09-20 12:00:00,1989-09-14 18:00:00,1989-09-21 18:00:00,...,995.22,-52.41,42.82,1001.76,-43.50,52.00,NaN,NaN,NaN,1989.0


In [13]:
# Function For Applying SLP Bounds
def ET_SLP_Bounds(Control_ET, RCP45_ET, RCP85_ET, Begin):
# Define SLP Bound For ET Begin and ET Complete
    Begin_SLP_Bound = 1008
    Compl_SLP_Bound = 1008
# Apply Bounds
    if Begin == True:
        Control_ET_Begin = Control_ET[Control_ET["ET Begin SLP"] <= Begin_SLP_Bound].reset_index()
        RCP45_ET_Begin = RCP45_ET[RCP45_ET["ET Begin SLP"] <= Begin_SLP_Bound].reset_index()
        RCP85_ET_Begin = RCP85_ET[RCP85_ET["ET Begin SLP"] <= Begin_SLP_Bound].reset_index()
        return (Control_ET_Begin, RCP45_ET_Begin, RCP85_ET_Begin)
    else:
        Control_ET_Compl = Control_ET[Control_ET["ET Complete SLP"] <= Compl_SLP_Bound].reset_index()
        RCP45_ET_Compl = RCP45_ET[RCP45_ET["ET Complete SLP"] <= Compl_SLP_Bound].reset_index()
        RCP85_ET_Compl = RCP85_ET[RCP85_ET["ET Complete SLP"] <= Compl_SLP_Bound].reset_index()
        return (Control_ET_Compl, RCP45_ET_Compl, RCP85_ET_Compl)

In [14]:
# Create Subsets For Storm Phase Cumulative Distributions
def Phase_Subsets(Data):
    Subset_0 = Data[Data["SLP(hPa)"] <= 1008].reset_index()
    Subset_Trop = Subset_0[Subset_0["Storm Phase"] == "Tropical"]
    Subset_Trans = Subset_0[Subset_0["Storm Phase"] == "Transition"]
    Subset_Extra = Subset_0[Subset_0["Storm Phase"] == "Extratropical"]
    return (Subset_0, Subset_Trop, Subset_Trans, Subset_Extra)

In [60]:
# Create Subsets For Phase Space Parameter Cumulative Distributions
def Param_Subsets(Data, Subset_Var):
    Subset_0 = Data[Data["SLP(hPa)"] <= 1008].reset_index()
    if Subset_Var == "SLP(hPa)":
        Subset_1 = Subset_0
        Subset_2 = Subset_0[(Subset_0[Subset_Var] > 0) & (Subset_0[Subset_Var] <= 1000)].reset_index()
        Subset_3 = Subset_0[(Subset_0[Subset_Var] > 0) & (Subset_0[Subset_Var] <= 990)].reset_index()
        Subset_4 = Subset_0[Subset_0[Subset_Var] <= 980].reset_index()
        Subset_Labels = ["SLP ≤ 1008hPa", "SLP ≤ 1000hPa", "SLP ≤ 990hPa", "SLP ≤ 980hPa"]
    elif Subset_Var == "Lat":
        Subset_1 = Subset_0[Subset_0[Subset_Var] >= 40].reset_index()
        Subset_2 = Subset_0[(Subset_0[Subset_Var] >= 35) & (Subset_0[Subset_Var] < 40)].reset_index()
        Subset_3 = Subset_0[(Subset_0[Subset_Var] >= 30) & (Subset_0[Subset_Var] < 35)].reset_index()
        Subset_4 = Subset_0[Subset_0[Subset_Var] < 30].reset_index()
        Subset_Labels = ["Lat >= 40N", "Lat 35~40N", "Lat 30~35N", "Lat < 30N"]
    elif Subset_Var == "Month":
        Subset_1 = Subset_0[(Subset_0[Subset_Var] >= 1) & (Subset_0[Subset_Var] <= 5)].reset_index()
        Subset_2 = Subset_0[(Subset_0[Subset_Var] >= 6) & (Subset_0[Subset_Var] <= 7)].reset_index()
        Subset_3 = Subset_0[(Subset_0[Subset_Var] >= 8) & (Subset_0[Subset_Var] <= 9)].reset_index()
        Subset_4 = Subset_0[(Subset_0[Subset_Var] >= 10) & (Subset_0[Subset_Var] <= 12)].reset_index()
        Subset_Labels = ["Jan-May", "Jun-Jul", "Aug-Sep", "Oct-Dec"]
    Subsets = [Subset_1, Subset_2, Subset_3, Subset_4]
    return (Subsets, Subset_Labels)

In [16]:
# Calculate 25%, Median, 75% Percentiles
def Percentile(Array):
    Percent_25 = round(numpy.nanpercentile(Array, 25), 1)
    Median = round(numpy.nanmedian(Array), 1)
    Percent_75 = round(numpy.nanpercentile(Array, 75), 1)
    return ([Percent_25, Median, Percent_75])

In [17]:
# Calculate Statistical Significance Using KS Test
def KS_Test(Control_Array, RCP45_Array, RCP85_Array):
    P_Val_RCP45 = round(stats.kstest(Control_Array, RCP45_Array)[1], 3)
    P_Val_RCP85 = round(stats.kstest(Control_Array, RCP85_Array)[1], 3)
    return (P_Val_RCP45, P_Val_RCP85)

In [18]:
# Create DataFrame to Store Percentiles Data
def Percentile_DF(Control_Array, RCP45_Array, RCP85_Array):
    Control_Percentiles = Percentile(Control_Array)
    RCP45_Percentiles = Percentile(RCP45_Array)
    RCP85_Percentiles = Percentile(RCP85_Array)
    P_Vals = KS_Test(Control_Array, RCP45_Array, RCP85_Array)
    Control_Percentiles.append(1.000)
    RCP45_Percentiles.append(P_Vals[0])
    RCP85_Percentiles.append(P_Vals[1])
    DF = pandas.DataFrame({"Percentile": ["25%", "Median", "75%", "P Val"], \
    "Control": Control_Percentiles, "RCP4.5": RCP45_Percentiles, "RCP8.5": RCP85_Percentiles})
    return (DF)

In [19]:
# Storm Peak Sea Level Pressure
Peak_SLP_DF = Percentile_DF(Control_ET["Peak SLP"], RCP45_ET["Peak SLP"], RCP85_ET["Peak SLP"])
Peak_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,948.4,948.800,949.900
1,Median,976.0,974.800,972.600
2,75%,988.1,988.500,987.500
3,P Val,1.0,0.946,0.573


In [20]:
# ET Initiation Sea Level Pressure
Control_ET_Begin, RCP45_ET_Begin, RCP85_ET_Begin = ET_SLP_Bounds(Control_ET, RCP45_ET, RCP85_ET, True)
ET_Begin_SLP_DF = Percentile_DF(Control_ET_Begin["ET Begin SLP"], RCP45_ET_Begin["ET Begin SLP"], RCP85_ET_Begin["ET Begin SLP"])
ET_Begin_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,980.9,979.50,980.900
1,Median,993.5,993.90,994.300
2,75%,1000.3,1000.00,1001.000
3,P Val,1.0,0.99,0.964


In [21]:
# ET Completion Sea Level Pressure
Control_ET_Compl, RCP45_ET_Compl, RCP85_ET_Compl = ET_SLP_Bounds(Control_ET, RCP45_ET, RCP85_ET, False)
ET_Complete_SLP_DF = Percentile_DF(Control_ET_Compl["ET Complete SLP"], RCP45_ET_Compl["ET Complete SLP"], RCP85_ET_Compl["ET Complete SLP"])
ET_Complete_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,988.5,986.800,984.600
1,Median,996.8,997.000,994.800
2,75%,1002.2,1001.800,1001.500
3,P Val,1.0,0.698,0.219


In [22]:
# Storm Peak Latitude
Peak_Lat_DF = Percentile_DF(Control_ET["Peak Lat"], RCP45_ET["Peak Lat"], RCP85_ET["Peak Lat"])
Peak_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,28.1,30.400,30.600
1,Median,34.5,37.500,37.600
2,75%,40.4,41.600,41.900
3,P Val,1.0,0.017,0.015


In [23]:
# ET Initiation Latitude
Control_ET_Begin, RCP45_ET_Begin, RCP85_ET_Begin = ET_SLP_Bounds(Control_ET, RCP45_ET, RCP85_ET, True)
ET_Begin_Lat_DF = Percentile_DF(Control_ET_Begin["ET Begin Lat"], RCP45_ET_Begin["ET Begin Lat"], RCP85_ET_Begin["ET Begin Lat"])
ET_Begin_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,33.4,33.900,34.200
1,Median,37.8,38.800,38.200
2,75%,42.2,42.700,42.400
3,P Val,1.0,0.519,0.191


In [24]:
# ET Completion Latitude
Control_ET_Compl, RCP45_ET_Compl, RCP85_ET_Compl = ET_SLP_Bounds(Control_ET, RCP45_ET, RCP85_ET, False)
ET_Complete_Lat_DF = Percentile_DF(Control_ET_Compl["ET Complete Lat"], RCP45_ET_Compl["ET Complete Lat"], RCP85_ET_Compl["ET Complete Lat"])
ET_Complete_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,38.3,39.500,38.800
1,Median,44.0,44.400,44.000
2,75%,47.6,47.900,47.600
3,P Val,1.0,0.896,0.611


In [25]:
# Storm Peak Longitude
Peak_Lon_DF = Percentile_DF(Control_ET["Peak Lon"], RCP45_ET["Peak Lon"], RCP85_ET["Peak Lon"])
Peak_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-72.2,-67.900,-67.400
1,Median,-60.0,-51.200,-57.000
2,75%,-42.6,-38.800,-40.000
3,P Val,1.0,0.002,0.083


In [26]:
# ET Initiation Longitude
Control_ET_Begin, RCP45_ET_Begin, RCP85_ET_Begin = ET_SLP_Bounds(Control_ET, RCP45_ET, RCP85_ET, True)
ET_Begin_Lon_DF = Percentile_DF(Control_ET_Begin["ET Begin Lon"], RCP45_ET_Begin["ET Begin Lon"], RCP85_ET_Begin["ET Begin Lon"])
ET_Begin_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-72.1,-68.2,-69.000
1,Median,-61.1,-50.3,-55.600
2,75%,-42.0,-37.8,-39.200
3,P Val,1.0,0.0,0.024


In [27]:
# ET Completion Longitude
Control_ET_Compl, RCP45_ET_Compl, RCP85_ET_Compl = ET_SLP_Bounds(Control_ET, RCP45_ET, RCP85_ET, False)
ET_Complete_Lon_DF = Percentile_DF(Control_ET_Compl["ET Complete Lon"], RCP45_ET_Compl["ET Complete Lon"], RCP85_ET_Compl["ET Complete Lon"])
ET_Complete_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-66.0,-59.700,-61.100
1,Median,-50.4,-42.600,-46.200
2,75%,-33.2,-30.300,-31.800
3,P Val,1.0,0.007,0.076


In [28]:
Control_All, Control_Trop, Control_Trans, Control_Extra = Phase_Subsets(Control_Data)
RCP45_All, RCP45_Trop, RCP45_Trans, RCP45_Extra = Phase_Subsets(RCP45_Data)
RCP85_All, RCP85_Trop, RCP85_Trans, RCP85_Extra = Phase_Subsets(RCP85_Data)

In [29]:
# Sea Level Pressure (All)
All_SLP_DF = Percentile_DF(Control_All["SLP(hPa)"], RCP45_All["SLP(hPa)"], RCP85_All["SLP(hPa)"])
All_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,978.9,977.9,977.5
1,Median,995.0,993.4,993.4
2,75%,1002.4,1001.4,1001.6
3,P Val,1.0,0.0,0.0


In [30]:
# Sea Level Pressure (Tropical)
Trop_SLP_DF = Percentile_DF(Control_Trop["SLP(hPa)"], RCP45_Trop["SLP(hPa)"], RCP85_Trop["SLP(hPa)"])
Trop_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,977.8,977.3,976.9
1,Median,995.6,993.8,993.6
2,75%,1002.8,1001.6,1002.0
3,P Val,1.0,0.0,0.0


In [31]:
# Sea Level Pressure (Transitioning)
Trans_SLP_DF = Percentile_DF(Control_Trans["SLP(hPa)"], RCP45_Trans["SLP(hPa)"], RCP85_Trans["SLP(hPa)"])
Trans_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,967.2,970.100,966.500
1,Median,987.0,987.400,986.300
2,75%,997.6,997.600,997.000
3,P Val,1.0,0.511,0.147


In [32]:
# Sea Level Pressure (Extratropical)
Extra_SLP_DF = Percentile_DF(Control_Extra["SLP(hPa)"], RCP45_Extra["SLP(hPa)"], RCP85_Extra["SLP(hPa)"])
Extra_SLP_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,990.0,986.0,987.2
1,Median,997.4,996.5,995.8
2,75%,1003.1,1002.6,1002.5
3,P Val,1.0,0.0,0.0


In [33]:
# Longitude (All)
All_Lon_DF = Percentile_DF(Control_All["Lon"], RCP45_All["Lon"], RCP85_All["Lon"])
All_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-72.8,-68.1,-70.2
1,Median,-57.9,-51.2,-54.6
2,75%,-40.4,-37.9,-38.8
3,P Val,1.0,0.0,0.0


In [34]:
# Longitude (Tropical)
Trop_Lon_DF = Percentile_DF(Control_Trop["Lon"], RCP45_Trop["Lon"], RCP85_Trop["Lon"])
Trop_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-74.4,-70.2,-73.1
1,Median,-60.5,-53.9,-56.5
2,75%,-42.6,-40.2,-40.5
3,P Val,1.0,0.0,0.0


In [35]:
# Longitude (Transitioning)
Trans_Lon_DF = Percentile_DF(Control_Trans["Lon"], RCP45_Trans["Lon"], RCP85_Trans["Lon"])
Trans_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-67.8,-65.5,-67.300
1,Median,-56.2,-49.1,-54.700
2,75%,-42.3,-37.2,-42.100
3,P Val,1.0,0.0,0.214


In [36]:
# Longitude (Extratropical)
Extra_Lon_DF = Percentile_DF(Control_Extra["Lon"], RCP45_Extra["Lon"], RCP85_Extra["Lon"])
Extra_Lon_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-62.7,-55.5,-61.100
1,Median,-45.0,-41.0,-41.800
2,75%,-27.3,-26.6,-26.600
3,P Val,1.0,0.0,0.014


In [37]:
# Latitude (All)
All_Lat_DF = Percentile_DF(Control_All["Lat"], RCP45_All["Lat"], RCP85_All["Lat"])
All_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,22.7,25.3,25.0
1,Median,30.8,33.4,33.4
2,75%,38.5,39.9,39.6
3,P Val,1.0,0.0,0.0


In [38]:
# Latitude (Tropical)
Trop_Lat_DF = Percentile_DF(Control_Trop["Lat"], RCP45_Trop["Lat"], RCP85_Trop["Lat"])
Trop_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,19.9,22.1,21.9
1,Median,27.2,30.4,30.1
2,75%,34.0,36.2,35.8
3,P Val,1.0,0.0,0.0


In [39]:
# Latitude (Transitioning)
Trans_Lat_DF = Percentile_DF(Control_Trans["Lat"], RCP45_Trans["Lat"], RCP85_Trans["Lat"])
Trans_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,33.6,34.700,35.8
1,Median,38.8,39.900,40.0
2,75%,43.4,43.600,43.4
3,P Val,1.0,0.014,0.0


In [40]:
# Latitude (Extratropical)
Extra_Lat_DF = Percentile_DF(Control_Extra["Lat"], RCP45_Extra["Lat"], RCP85_Extra["Lat"])
Extra_Lat_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,41.2,39.400,38.8
1,Median,46.2,45.400,44.9
2,75%,49.8,49.800,49.6
3,P Val,1.0,0.004,0.0


In [61]:
Control_Subsets, Subset_Labels = Param_Subsets(Control_Data, "SLP(hPa)")
RCP45_Subsets, Subset_Labels = Param_Subsets(RCP45_Data, "SLP(hPa)")
RCP85_Subsets, Subset_Labels = Param_Subsets(RCP85_Data, "SLP(hPa)")

In [62]:
# B (<= 1008hPa)
B_1008hPa_DF = Percentile_DF(Control_Subsets[0]["B"], RCP45_Subsets[0]["B"], RCP85_Subsets[0]["B"])
B_1008hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-2.8,-1.9,-1.6
1,Median,4.4,6.0,6.7
2,75%,16.2,16.9,17.6
3,P Val,1.0,0.0,0.0


In [63]:
# VLT (<= 1008hPa)
VLT_1008hPa_DF = Percentile_DF(Control_Subsets[0]["VLT"], RCP45_Subsets[0]["VLT"], RCP85_Subsets[0]["VLT"])
VLT_1008hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,39.9,33.800,23.0
1,Median,113.3,104.400,101.2
2,75%,205.2,193.500,192.4
3,P Val,1.0,0.001,0.0


In [64]:
# B (<= 1000hPa)
B_1000hPa_DF = Percentile_DF(Control_Subsets[1]["B"], RCP45_Subsets[1]["B"], RCP85_Subsets[1]["B"])
B_1000hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-2.3,-1.50,-0.7
1,Median,6.2,6.80,7.7
2,75%,18.4,17.90,18.9
3,P Val,1.0,0.01,0.0


In [65]:
# VLT (<= 1000hPa)
VLT_1000hPa_DF = Percentile_DF(Control_Subsets[1]["VLT"], RCP45_Subsets[1]["VLT"], RCP85_Subsets[1]["VLT"])
VLT_1000hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,65.5,58.9,46.7
1,Median,165.6,141.8,139.9
2,75%,241.2,225.2,223.8
3,P Val,1.0,0.0,0.0


In [66]:
# B (<= 990hPa)
B_990hPa_DF = Percentile_DF(Control_Subsets[2]["B"], RCP45_Subsets[2]["B"], RCP85_Subsets[2]["B"])
B_990hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-2.6,-1.500,-0.5
1,Median,5.8,6.900,7.8
2,75%,18.1,18.600,19.9
3,P Val,1.0,0.001,0.0


In [67]:
# VLT (<= 990hPa)
VLT_990hPa_DF = Percentile_DF(Control_Subsets[2]["VLT"], RCP45_Subsets[2]["VLT"], RCP85_Subsets[2]["VLT"])
VLT_990hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,133.9,89.4,85.7
1,Median,220.0,190.0,187.6
2,75%,273.2,253.5,250.2
3,P Val,1.0,0.0,0.0


In [68]:
# B (<= 980hPa)
B_980hPa_DF = Percentile_DF(Control_Subsets[3]["B"], RCP45_Subsets[3]["B"], RCP85_Subsets[3]["B"])
B_980hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,-3.4,-2.100,-0.9
1,Median,4.3,5.200,6.5
2,75%,16.0,17.200,18.3
3,P Val,1.0,0.001,0.0


In [69]:
# VLT (<= 980hPa)
VLT_980hPa_DF = Percentile_DF(Control_Subsets[3]["VLT"], RCP45_Subsets[3]["VLT"], RCP85_Subsets[3]["VLT"])
VLT_980hPa_DF

,Percentile,Control,RCP4.5,RCP8.5
0,25%,198.2,147.6,136.7
1,Median,249.8,227.9,221.5
2,75%,290.4,272.1,265.3
3,P Val,1.0,0.0,0.0
